In [1]:
import pandas as pd
import os
import indian_state_list as isl
import tkinter as tk
import indian_map_image_clor as in_map
import time
import threading

d_state = isl.d_state
path = os.getcwd()

year = 2010

database = pd.read_csv(path + '/year_record_filtered/record_'+ str(year) +'_filtered.csv')
database = database.iloc[:,1:]
database.head()

,SUBDIVISION,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANNUAL,JF,MAM,JJAS,OND
0,andhra pradesh,2010,21.8,2.30,4.4,14.80,162.0,156.1,318.90,248.60,230.50,204.0,210.9,138.20,1712.40,24.10,181.20,954.00,553.10
1,arunachal pradesh,2010,0.6,13.20,237.8,466.90,312.7,509.9,378.00,321.50,444.20,97.7,58.9,14.20,2855.70,13.90,1017.50,1653.60,170.90
2,assam,2010,0.5,3.10,99.6,389.30,393.4,569.7,415.10,387.00,318.30,116.2,11.6,7.60,2711.50,3.70,882.30,1690.20,135.30
3,west bengal,2010,2.9,14.95,42.8,96.25,231.7,373.7,394.85,326.95,250.35,81.2,7.1,15.95,1838.55,17.85,370.65,1345.75,104.25
4,bihar,2010,0.1,1.10,0.2,6.50,44.4,80.9,191.20,158.40,105.70,33.6,5.0,2.10,629.20,1.30,51.10,536.20,40.70


In [2]:
def update_year_dataframe():
    global year, database
    database = pd.read_csv(path + '/year_record_filtered/record_'+ str(year) +'_filtered.csv')
    database = database.iloc[:,1:]

def get_value_month(state,month) :
    d = database[database.SUBDIVISION == state].copy()
    return d.iloc[:,month]

def get_max_value_month(month) :
    return database.iloc[:,month].max()

def convert_to_color(state,month):
    month = month + 1
    max_v = get_max_value_month(month)
    div = 10
    element = max_v // div
    val = get_value_month(state,month)
    c = val // element
    color = (int(250 - (200 / div * c)), 0, 0)
    return color

def create_color_list(d_state,month):
    color_list = []
    for x in d_state.keys():
        color_list.append(convert_to_color(x,month))
    return color_list

def create_color_rbg_2_hex(div):
    l = []
    for c in range(div):
        r,g,b = (int(250 - (200 // div * c)), 0, 0)
        l.append("#{:02x}{:02x}{:02x}".format(r,g,b))
    return l

print(create_color_list(d_state,1))
print(create_color_rbg_2_hex(10))

[(250, 0, 0), (250, 0, 0), (250, 0, 0), (50, 0, 0), (250, 0, 0), (110, 0, 0), (250, 0, 0), (190, 0, 0), (210, 0, 0), (210, 0, 0), (190, 0, 0), (250, 0, 0), (250, 0, 0), (250, 0, 0), (250, 0, 0), (250, 0, 0), (90, 0, 0), (190, 0, 0), (230, 0, 0), (230, 0, 0), (130, 0, 0), (250, 0, 0), (250, 0, 0), (250, 0, 0), (250, 0, 0), (250, 0, 0), (230, 0, 0), (250, 0, 0), (230, 0, 0)]
['#fa0000', '#e60000', '#d20000', '#be0000', '#aa0000', '#960000', '#820000', '#6e0000', '#5a0000', '#460000']


In [3]:
window = tk.Tk()
window.title('Rain Visualisation (India 1901 - 2017)')
w = 1000
h = 750

month_selected = 1
map_drawn = ""

map_IN = None

year_selected =tk.StringVar(window)

def loading_window():
    global canvas_loading
    canvas_loading.config(width=w, height=h)
    time.sleep(2)
    canvas_loading.config(width=0, height=0)

def strat_loading():
    threading.Thread(target=loading_window).start()

def create_color_rbg_2_hex(div):
    l = []
    for c in range(div):
        b,g,r = (int(250 - (200 // div * c)), 0, 0)
        l.append("#{:02x}{:02x}{:02x}".format(r,g,b))
    return l

def draw_map_given(month) :
    return in_map.create_map_india_with_state(create_color_list(d_state,month))

def select_month_btns(event):
    global month_selected,map_drawn,map_IN,parent_canvas,img
    case = event.x // 50 + 1
    if event.y > 50 :
        case = case + 6
    print(case)
    strat_loading()
    if month_selected == case :
        print("Same Selection")
    else :
        canvas_month_sel.itemconfig('month_btn' + str(month_selected), fill="#666666", outline="")
        month_selected = case
        canvas_month_sel.itemconfig('month_btn' + str(month_selected), fill="#000096", outline="")
        map_drawn = draw_map_given(case)
        img = tk.PhotoImage(file=map_drawn, master=parent_canvas)
        parent_canvas.itemconfig('map_IN' ,image=img)
        update_legends(get_max_value_month(case+1))
        
def select_year_btn(event):
    global year, month_selected,img,parent_canvas,map_drawn
    year = year_selected.get()
    strat_loading()
    if 1901 <= int(year) <= 2017:   
        update_year_dataframe()
        canvas_month_sel.itemconfig('month_btn' + str(month_selected), fill="#666666", outline="")
        month_selected = 1
        canvas_month_sel.itemconfig('month_btn' + str(month_selected), fill="#000096", outline="")
        map_drawn = draw_map_given(1)
        img = tk.PhotoImage(file=map_drawn, master=parent_canvas)
        parent_canvas.itemconfig('map_IN' ,image=img)
        update_legends(get_max_value_month(1))
        parent_canvas.itemconfig("change_year_txt", text="Current Year : "+ str(year) +" Change Year")
    else :
        parent_canvas.itemconfig("change_year_txt", text="Invalid Year Try Again.")
    
def update_legends(max_v) :
    element_val = max_v / 10
    for x in range(10):
        canvas_legends.itemconfig( 'legend_txt'+str(x), text="- " + str(round(((x + 1) * element_val),1))+" mm")
        
# map resolution is constant 662 x 750
parent_canvas = tk.Canvas(window, width=w, height=h, bg="#dcdcdc", highlightthickness=0)

canvas_month_sel = tk.Canvas(parent_canvas, width=310, height=100, bg="#dddddd", highlightthickness=0)
for x in range(6):
    btn1 = canvas_month_sel.create_oval( 10 + x * 50, 10, 50 + x * 50, 50, fill="#666666", outline="", tags="month_btn" + str(x + 1))
    txt1 = canvas_month_sel.create_text( 30 + x * 50, 28, text=(x + 1), font='Time 20 bold', fill="#ffffff", tags="month_txt" + str(x + 1))
    canvas_month_sel.tag_bind(btn1, '<Button-1>', select_month_btns)
    canvas_month_sel.tag_bind(txt1, '<Button-1>', select_month_btns)
    btn2 = canvas_month_sel.create_oval( 10 + x * 50, 60, 50 + x * 50, 100, fill="#666666", outline="", tags="month_btn" + str(x + 7))
    txt2 = canvas_month_sel.create_text( 30 + x * 50, 78, text=(x + 7), font='Time 20 bold', fill="#ffffff", tags="month_txt" + str(x + 7))
    canvas_month_sel.tag_bind(btn2, '<Button-1>', select_month_btns)
    canvas_month_sel.tag_bind(txt2, '<Button-1>', select_month_btns)
else :
    canvas_month_sel.itemconfig('month_btn1', fill="#000096", outline="")
    map_drawn = draw_map_given(1)
parent_canvas.create_window(675,500, anchor=tk.NW, window=canvas_month_sel)

canvas_year_sel = tk.Canvas(parent_canvas, width=300, height=100, bg="#dddddd", highlightthickness=0)
parent_canvas.create_window(710,680, anchor=tk.NW, window=canvas_year_sel)

year_entry = tk.Entry(canvas_year_sel, textvariable=year_selected, justify='center')
canvas_year_sel.create_window(0,10, anchor=tk.NW, window=year_entry)
btn2 = canvas_year_sel.create_oval( 200, 0, 250, 50, fill="#666666", outline="", tags="year_btn")
txt2 = canvas_year_sel.create_text( 225, 23, text="Go", font='Time 20 bold', fill="#ffffff", tags="year_txt")
canvas_year_sel.tag_bind(btn2, '<Button-1>', select_year_btn)
canvas_year_sel.tag_bind(txt2, '<Button-1>', select_year_btn)

canvas_legends = tk.Canvas(parent_canvas, width=100, height=330,  bg="#dcdcdc", highlightthickness=0)
parent_canvas.create_window(10,420, anchor=tk.NW, window=canvas_legends)
legend_color = create_color_rbg_2_hex(10)
element_val = get_max_value_month(2)/10
print(get_max_value_month(2))
canvas_legends.create_text(29, 0, text= "- 00.0 mm", font='Time 12', fill="#666666", anchor=tk.NW)
for x in range(10):
    canvas_legends.create_line(10, 20 + x * 30 ,30, 20 + x * 30, width=30, fill=legend_color[x])
    canvas_legends.create_text(29, 24 + x * 30, text= "- " + str(round(((x + 1) * element_val),1))+" mm", font='Time 12', fill="#666666", tags="legend_txt"+str(x), anchor=tk.NW)

img = tk.PhotoImage(file=map_drawn, master=parent_canvas)
map_IN = parent_canvas.create_image(100, 0, image=img, anchor = tk.NW, tags="map_IN")
parent_canvas.create_text( 830 , 458, text="Select the Month from given below", font='Time 20', fill="#222222")
parent_canvas.create_text( 830 , 478, text="Buttons.", font='Time 20', fill="#222222")
parent_canvas.create_text( 830 , 650, text="Current Year : "+ str(year) +" Change Year", font='Time 20', fill="#222222", tags="change_year_txt")
canvas_loading = tk.Canvas(parent_canvas, width=0, height=0, bg="#dcdcdc", highlightthickness=0)
parent_canvas.create_window(0, 0, window=canvas_loading, anchor=tk.NW)
parent_canvas.pack()
tk.mainloop()

750 662
<class 'list'> 381
[220 220 220]
30.9
750 662
<class 'list'> 381
[220 220 220]
10
750 662
<class 'list'> 381
[220 220 220]
7
750 662
<class 'list'> 381
[220 220 220]
